<center><h1>Automated ML</h1></center>

### Import Dependencies. 

In [11]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.interpret import ExplanationClient
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice
from azureml.core import Webservice

from azureml.automl.runtime.onnx_convert import OnnxConverter

import pandas as pd

import time
import logging
import requests
import json
import sys

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'HotelBookingDemand'

experiment=Experiment(ws, experiment_name)

In [3]:
cpu_cluster_name = "udacity-casptone-project"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
#dataset = Dataset.get_by_name(ws, name='hotel_booking')
dataset = Dataset.get_by_name(ws, name='hotel_booking_cleaned')

training_data, validation_data = dataset.random_split(percentage=0.8, seed=1)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "name": "AutoML_HotelBookingDemand_Experiment_{0}".format(time.time()),
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 15,
    "primary_metric_name": 'accuracy',
    "max_concurrent_iterations": 5
}

automl_config = AutoMLConfig(task = 'classification',                                                          
                             training_data=training_data,
                             validation_data= validation_data,
                             label_column_name = 'is_canceled',  
                             enable_onnx_compatible_models=True,
                             compute_target=cpu_cluster,                           
                                **automl_settings
                                )                              

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=False)

Running on remote.
No run_configuration provided, running on udacity-casptone-project with default configuration
Running on remote compute: udacity-casptone-project
Parent Run ID: AutoML_5b71b2ed-e916-4716-883d-5b5922f065a7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**********************************************************************************

         1   MaxAbsScaler XGBoostClassifier                 0:01:01       0.8494    0.8494
         2   MaxAbsScaler RandomForest                      0:00:57       0.7720    0.8494
         3   MaxAbsScaler RandomForest                      0:00:56       0.7059    0.8494
         4   MaxAbsScaler RandomForest                      0:00:55       0.7971    0.8494
         0   MaxAbsScaler LightGBM                          0:01:04       0.8711    0.8711
         5   MaxAbsScaler RandomForest                      0:01:03       0.7936    0.8711
         6   SparseNormalizer XGBoostClassifier             0:01:36       0.8630    0.8711
         7   SparseNormalizer XGBoostClassifier             0:01:37       0.8584    0.8711
         8   SparseNormalizer XGBoostClassifier             0:01:25       0.8627    0.8711
         9   SparseNormalizer XGBoostClassifier             0:01:10       0.8526    0.8711
        11   SparseNormalizer XGBoostClassifier             0:00:59       0.7644    0.8711

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output = False)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Get your best run and save the model from that run.
best_run, fitted_model = remote_run.get_output(metric = "accuracy")
print(best_run)

description = 'Best AutoML Model'
tags = None

model = remote_run.register_model(description = description, tags = tags)
print(remote_run.model_id)

Run(Experiment: HotelBookingDemand,
Id: AutoML_5b71b2ed-e916-4716-883d-5b5922f065a7_37,
Type: azureml.scriptrun,
Status: Completed)
AutoML5b71b2ede37


In [9]:
#TODO: Save the best model
print('fitted_model:\n {}\n\n'.format(fitted_model))

fitted_model:
 Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.01,
                                                                                                    min_weight_fraction_leaf=0.0,


In [12]:
featurizer = fitted_model.named_steps['datatransformer']
df_f = featurizer.get_featurization_summary()
pd.DataFrame(data=df_f)

,RawFeatureName,TypeDetected,Dropped,EngineeredFeatureCount,Transformations
0,hotel,Categorical,No,1,[ModeCatImputer-StringCast-LabelEncoder]
1,stays_in_weekend_nights,Categorical,No,16,[StringCast-CharGramCountVectorizer]
2,stays_in_week_nights,Categorical,No,31,[StringCast-CharGramCountVectorizer]
3,adults,Categorical,No,13,[StringCast-CharGramCountVectorizer]
4,children,Categorical,No,5,[StringCast-CharGramCountVectorizer]
5,babies,Categorical,No,4,[StringCast-CharGramCountVectorizer]
6,meal,Categorical,No,5,[StringCast-CharGramCountVectorizer]
7,country,Categorical,No,172,[StringCast-CharGramCountVectorizer]
8,market_segment,Categorical,No,8,[StringCast-CharGramCountVectorizer]
9,distribution_channel,Categorical,No,5,[StringCast-CharGramCountVectorizer]


In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
HotelBookingDemand,AutoML_5b71b2ed-e916-4716-883d-5b5922f065a7_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
client = ExplanationClient.from_run(best_run)
global_explanation = client.download_model_explanation(top_k=5)
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [15]:
global_explanation_topk = client.download_model_explanation(top_k=5)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [16]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [0.8247471211665076, 0.6750913708132136, 0.503033341166726, 0.34331272130901025, 0.2938431708255052]
global importance names: ['deposit_type', 'country', 'market_segment', 'total_of_special_requests', 'lead_time']


### ONNX Model

In [18]:
onnx_best_run, onnx_model = remote_run.get_output(return_onnx_model=True)

onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_fl_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
best_run_metrics = best_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

mymodel = best_run.register_model(model_name='hoteldemand_automl', model_path='outputs/model.pkl', description = description, tags = tags, properties={'accuracy': best_run_metrics['accuracy']})

f1_score_macro 0.8628372427969981
balanced_accuracy 0.8562668268837611
f1_score_weighted 0.872699410487172
precision_score_micro 0.8740585774058578
AUC_macro 0.9443973591448382
f1_score_micro 0.8740585774058578
weighted_accuracy 0.8897353740238628
log_loss 0.3055704363846601
average_precision_score_macro 0.9428824457949974
precision_score_weighted 0.8736151821483125
accuracy 0.8740585774058578
AUC_micro 0.9493440906146601
average_precision_score_weighted 0.9485711482406793
AUC_weighted 0.9443973591448384
recall_score_micro 0.8740585774058578
precision_score_macro 0.8720065355859143
norm_macro_recall 0.7125336537675222
recall_score_weighted 0.8740585774058578
average_precision_score_micro 0.9515296051299924
matthews_correlation 0.7281032564498381
recall_score_macro 0.8562668268837611
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_5b71b2ed-e916-4716-883d-5b5922f065a7_37/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_5b71b2ed-e916-4716-883d-5b5922f06

In [20]:
service_name = "myautomlbestrunv3" # Services Name

myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")
myenv.register(workspace=ws)

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv) # We are going to use score.py from AML

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores =2, memory_gb =4, auth_enabled=True, enable_app_insights=True) #Enable logging


# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = service_name,
    models = [mymodel],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
print(ws.webservices)

service = Webservice(ws, service_name)
print(service.get_logs())

{'myautomlbestrunv3': AciWebservice(workspace=Workspace.create(name='Experiments', subscription_id='c643ea10-d987-475a-a877-13474b748947', resource_group='VioxAI'), name=myautomlbestrunv3, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=http://a9abe1f5-042c-4c29-b4f3-418d0a74cf84.eastus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})}
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version informati

In [22]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

primary, secondary = service.get_keys()
print(primary)

Healthy
http://a9abe1f5-042c-4c29-b4f3-418d0a74cf84.eastus.azurecontainer.io/score
http://a9abe1f5-042c-4c29-b4f3-418d0a74cf84.eastus.azurecontainer.io/swagger.json
juSpQfBcUXNs4My4WUiBxqDRba8iFdAY


In [46]:
scoring_uri = service.scoring_uri
key = primary

data = {
    "data": [
     {
          "hotel": "Resort Hotel",
          "lead_time": 85,
          "stays_in_weekend_nights": 0,
          "stays_in_week_nights": 3,
          "adults": 2,
          "children": 0,
          "babies": 0,
          "meal": "BB",
          "country": "PRT",
          "market_segment": "Online TA",
          "distribution_channel": "TA/TO",
          "is_repeated_guest": 0,
          "booking_changes": 0,
          "deposit_type": "No Deposit",
          "agent": 240,
          "company": 0,
          "days_in_waiting_list": 0,
          "customer_type": "Trasient",
          "adr": 82,
          "required_car_parking_spaces": 1,
          "total_of_special_requests": 1,
          "room": 1,
          "net_cancelled": 0
     }
   ]
}

    
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

headers = {"Content-Type": "application/json"}
    
headers["Authorization"] = f"Bearer {key}"

resp = requests.post(scoring_uri, input_data, headers=headers)
result = json.loads(resp.json())

print("Will the hotel booking be canceled? {}".format("Yes" if int(result['result'][0]) == 1 else "No"))

Will the hotel booking be canceled? No


In [47]:
print(service.get_logs()) #logs

/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-11T15:12:03,039768240+00:00 - gunicorn/run 
2021-01-11T15:12:03,039941141+00:00 - iot-server/run 
2021-01-11T15:12:03,041118246+00:00 - rsyslog/run 
2021-01-11T15:12:03,042288352+00:00 - nginx/run 
rsyslogd

In [48]:
service.delete()